In [ ]:
# ==================================
# Shell.ai 2025 - 08_modeling_prop1_prop7_boosted_STACKED_PERFORMANCE.py
# ==================================

import pandas as pd  # Data handling
import numpy as np  # Numerical computations
import joblib  # For saving/loading preprocessed data and models
import os  # File path operations
import warnings  # Suppress warnings
from sklearn.model_selection import KFold  # Cross-validation
from lightgbm import LGBMRegressor  # Gradient boosting base model
from lightgbm.callback import early_stopping  # Early stopping in LGBM
from sklearn.feature_selection import SelectFromModel  # Feature selection
from sklearn.preprocessing import PowerTransformer  # Target transformation
from sklearn.decomposition import PCA  # Dimensionality reduction
from sklearn.cluster import KMeans  # Clustering for feature
from sklearn.ensemble import StackingRegressor  # Stacked ensemble
from sklearn.linear_model import RidgeCV  # Linear regression with CV for final estimator

# Suppress warnings for cleaner notebook/log outputs
warnings.filterwarnings('ignore')

# =============================
#  Custom Shell MAPE Metric
# =============================
def shell_mape(y_true, y_pred, epsilon=1e-6):
    """
    Compute Mean Absolute Percentage Error with epsilon to avoid division by zero.
    Ignores near-zero true values for stability.
    """
    y_true = np.asarray(y_true)  # Ensure numpy arrays
    y_pred = np.asarray(y_pred)
    non_zero_mask = np.abs(y_true) > epsilon  # Ignore very small true values
    if not np.any(non_zero_mask):  # Edge case: all values effectively zero
        return np.inf
    mape = np.mean(
        np.abs((y_true[non_zero_mask] - y_pred[non_zero_mask]) /
               (np.abs(y_true[non_zero_mask]) + epsilon))
    )
    return mape

# =============================
#  Feature Engineering Functions
# =============================
def add_blend_weighted_features(df):
    """Compute weighted sum of component properties by blend fractions."""
    df_copy = df.copy()
    for j in range(1, 11):  # For each target property
        df_copy[f"BlendWeighted_Property{j}"] = 0.0  # Initialize column
        for i in range(1, 6):  # For each component
            blend_col = f"Component{i}_fraction"
            prop_col = f"Component{i}_Property{j}"
            if blend_col in df_copy.columns and prop_col in df_copy.columns:
                df_copy[f"BlendWeighted_Property{j}"] += df_copy[blend_col] * df_copy[prop_col]
    return df_copy

def add_nonlinear_blend_transforms(df):
    """Apply nonlinear transformations (sq, cube, sqrt, log, inv) to blend fractions."""
    df_copy = df.copy()
    for i in range(1, 6):
        col = f"Component{i}_fraction"
        if col in df_copy.columns:
            df_copy[f"{col}_sq"] = df_copy[col] ** 2
            df_copy[f"{col}_sqrt"] = np.sqrt(df_copy[col].clip(lower=0))  # Avoid negative sqrt
            df_copy[f"{col}_log"] = np.log1p(df_copy[col].clip(lower=1e-6))  # Avoid log(0)
            df_copy[f"{col}_cube"] = df_copy[col] ** 3
            df_copy[f"{col}_inv"] = 1 / (df_copy[col] + 1e-6)  # Avoid div by zero
    return df_copy

def add_interaction_terms(df):
    """Add pairwise interaction terms between blend fractions."""
    df_copy = df.copy()
    cols = [f"Component{i}_fraction" for i in range(1, 6)]
    cols = [c for c in cols if c in df_copy.columns]  # Only existing columns
    for i in range(len(cols)):
        for j in range(i + 1, len(cols)):
            df_copy[f"{cols[i].replace('fraction', '')}_x{cols[j].replace('_fraction', '')}"] = df_copy[cols[i]] * df_copy[cols[j]]
    return df_copy

def add_pca_features(df, n_components=3):
    """Reduce blend fraction dimensions using PCA and add as features."""
    df_copy = df.copy()
    cols = [f"Component{i}_fraction" for i in range(1, 6)]
    cols = [c for c in cols if c in df_copy.columns]
    if not cols or df_copy[cols].empty:  # Skip if no relevant columns
        return df_copy
    temp_df = df_copy[cols].copy().fillna(0)
    n_components = min(n_components, temp_df.shape[1])  # Adjust if fewer columns
    if n_components > 0 and temp_df.shape[0] > 0:
        pca = PCA(n_components=n_components, random_state=42)
        pca_features = pca.fit_transform(temp_df)
        for i in range(pca_features.shape[1]):
            df_copy[f"pca_{i}"] = pca_features[:, i]
    return df_copy

def add_kmeans_cluster_feature(df, n_clusters=5):
    """Cluster blend fractions and add cluster label as a feature."""
    df_copy = df.copy()
    comp_cols = [f"Component{i}_fraction" for i in range(1, 6)]
    comp_cols = [c for c in comp_cols if c in df_copy.columns]
    if not comp_cols or df_copy[comp_cols].empty:
        df_copy["blend_cluster"] = -1
        return df_copy
    temp_df = df_copy[comp_cols].copy().fillna(0)
    n_clusters = min(n_clusters, temp_df.shape[0] // 5 + 1)  # Ensure at least ~5 samples per cluster
    if n_clusters < 2:
        n_clusters = 2
    try:
        kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
        df_copy["blend_cluster"] = kmeans.fit_predict(temp_df)
    except ValueError:  # Fallback for very small datasets
        df_copy["blend_cluster"] = -1
    return df_copy

# =============================
#  Load Processed Data
# =============================
print("\n📅 Loading processed training and test data")
X_y = joblib.load("data/processed/train_processed.pkl")  # Preprocessed train
X_test = joblib.load("data/processed/test_processed.pkl")  # Preprocessed test
X_raw, y = X_y
X_test_raw = X_test

if isinstance(y, pd.Series):
    y = y.to_frame()  # Ensure multi-output consistency

print(f"Train X (raw): {X_raw.shape}")
print(f"Train y: {y.shape}")
print(f"Test X (raw): {X_test_raw.shape}")

# Copy for processing
X_processed = X_raw.copy()
X_test_processed = X_test_raw.copy()

# =============================
#  Apply Feature Engineering
# =============================
print("Applying feature engineering...")
for func in [add_blend_weighted_features, add_nonlinear_blend_transforms,
             add_interaction_terms, add_pca_features, add_kmeans_cluster_feature]:
    X_processed = func(X_processed)
    X_test_processed = func(X_test_processed)

# Fill missing values post-feature engineering
X_processed.fillna(0, inplace=True)
X_test_processed.fillna(0, inplace=True)

# =============================
#  Align Columns Between Train/Test
# =============================
train_cols = set(X_processed.columns)
test_cols = set(X_test_processed.columns)
common_cols = list(train_cols.intersection(test_cols))

# Add missing columns in train/test to maintain alignment
for col in (train_cols - test_cols):
    X_test_processed[col] = 0.0
for col in (test_cols - train_cols):
    X_processed[col] = 0.0

# Ensure consistent column order
X_processed = X_processed[common_cols].reindex(columns=sorted(common_cols))
X_test_processed = X_test_processed[common_cols].reindex(columns=sorted(common_cols))

# Convert object columns to numeric
for col in X_processed.columns:
    X_processed[col] = pd.to_numeric(X_processed[col], errors='coerce')
    X_test_processed[col] = pd.to_numeric(X_test_processed[col], errors='coerce')

# Fill remaining missing values
X_processed.fillna(0, inplace=True)
X_test_processed.fillna(0, inplace=True)

X_processed_np = X_processed.values  # Convert to NumPy for model
X_test_processed_np = X_test_processed.values

# =============================
#  Cross-validation and Stacking Setup
# =============================
kf = KFold(n_splits=5, shuffle=True, random_state=42)  # 5-fold CV
seeds = [42, 2024]  # Multiple seeds for model ensembling

cv_total = []  # Store per-target CV scores
target_cols = y.columns.tolist()
test_preds_all = np.zeros((X_test_processed_np.shape[0], len(target_cols)))  # Store final test preds

special_targets = ["BlendProperty1", "BlendProperty4", "BlendProperty5",
                   "BlendProperty7", "BlendProperty9"]  # Could be used for extra tuning
es_callback = early_stopping(stopping_rounds=70, verbose=False)  # Early stopping LGBM

# =============================
#  Modeling Loop per Target
# =============================
for t_idx, target in enumerate(target_cols):
    print(f"\n Target: {target}")

    pt = PowerTransformer(method='yeo-johnson')  # Stabilize target distribution
    y_trans = pt.fit_transform(y[[target]])[:, 0]  # Transform target

    # Feature selection using LightGBM
    selector_model = LGBMRegressor(n_estimators=150, random_state=0, n_jobs=-1)
    selector_model.fit(X_processed_np, y[target])
    max_feats = 120 if target == "BlendProperty1" else (100 if target == "BlendProperty7" else 60)
    selector = SelectFromModel(selector_model, prefit=True, max_features=max_feats, threshold=-np.inf)
    X_sel = selector.transform(X_processed_np)
    X_test_sel = selector.transform(X_test_processed_np)
    print(f"Selected initial features shape: {X_sel.shape}")

    oof_preds = np.zeros(X_processed_np.shape[0])  # Out-of-fold predictions
    test_preds = np.zeros(X_test_processed_np.shape[0])

    # Seed ensembling
    for seed in seeds:
        fold_preds = np.zeros(X_processed_np.shape[0])
        fold_test = np.zeros(X_test_processed_np.shape[0])

        for fold, (tr_idx, val_idx) in enumerate(kf.split(X_sel)):
            X_tr, X_val = X_sel[tr_idx], X_sel[val_idx]
            y_tr, y_val = y_trans[tr_idx], y_trans[val_idx]

            # Base models for stacking
            lgbm_base_model = LGBMRegressor(
                n_estimators=500,
                learning_rate=0.02,
                max_depth=5,
                num_leaves=32,
                min_child_samples=20,
                subsample=0.7,
                reg_alpha=0.1,
                reg_lambda=0.1,
                random_state=seed,
                n_jobs=-1
            )

            estimators = [
                ('lgbm', lgbm_base_model),
                ('ridge', RidgeCV(alphas=np.logspace(-3, 3, 7), cv=kf))  # Meta-features from Ridge
            ]

            # Stacking ensemble
            stack_model = StackingRegressor(
                estimators=estimators,
                final_estimator=RidgeCV(alphas=np.logspace(-3, 3, 3)),  # Final linear blend
                cv=kf,
                n_jobs=-1
            )

            # Fit stacking model
            stack_model.fit(X_tr, y_tr)

            # Store fold predictions
            fold_preds[val_idx] = stack_model.predict(X_val)
            fold_test += stack_model.predict(X_test_sel) / kf.get_n_splits()

        # Average across seeds
        oof_preds += fold_preds / len(seeds)
        test_preds += fold_test / len(seeds)

    # Inverse transform predictions to original scale
    oof_preds_inv = pt.inverse_transform(oof_preds.reshape(-1, 1))[:, 0]
    test_preds_inv = pt.inverse_transform(test_preds.reshape(-1, 1))[:, 0]

    # Clip extreme predictions to avoid unrealistic values
    q_low, q_high = y[target].quantile([0.005, 0.995])
    test_preds_inv = np.clip(test_preds_inv, q_low, q_high)
    oof_preds_inv = np.clip(oof_preds_inv, q_low, q_high)

    cv_score = shell_mape(y[target], oof_preds_inv)  # CV metric
    print(f" CV MAPE: {cv_score:.6f}")

    test_preds_all[:, t_idx] = test_preds_inv  # Store test predictions
    cv_total.append(cv_score)  # Store CV score

# =============================
#  CV Summary
# =============================
print("\n====================")
print(" CV MAPE per target:")
for col, score in zip(target_cols, cv_total):
    print(f"{col}: {score:.6f}")
print(f"\n Avg CV MAPE: {np.mean(cv_total):.6f}")

# =============================
#  Save Submission
# =============================
os.makedirs("submissions", exist_ok=True)  # Ensure folder exists
script_dir = os.path.dirname(__file__) if '__file__' in locals() else os.getcwd()
test_csv_paths = [
    os.path.join(script_dir, "data", "raw", "test.csv"),
    os.path.join(script_dir, "..", "data", "raw", "test.csv"),
    os.path.join(script_dir, "..", "..", "data", "raw", "test.csv"),
    "data/raw/test.csv",
]

# Load raw test CSV
test_raw = None
for path in test_csv_paths:
    if os.path.exists(path):
        test_raw = pd.read_csv(path)
        break
if test_raw is None:
    raise FileNotFoundError("test.csv not found in expected locations.")

id_col = "ID" if "ID" in test_raw.columns else test_raw.columns[0]

# Prepare submission
submission = pd.DataFrame({id_col: test_raw[id_col]})
for i, col in enumerate(target_cols):
    submission[col] = test_preds_all[:, i]

submission.to_csv("submissions/prop1_prop7_boosted_STACKED_PERFORMANCE_submission.csv", index=False)
print("\n Submission saved to submissions/prop1_prop7_boosted_STACKED_PERFORMANCE_submission.csv")


: 